In [6]:
import numpy as np
import pandas as pd
import os 
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [4]:
load_dotenv()

url = URL.create(
    drivername="postgresql+psycopg2",
    username=os.getenv("NEON_USER"),
    password=os.getenv("NEON_PASSWORD"),
    host=os.getenv("NEON_HOST"),
    port=int(os.getenv("NEON_PORT", "5432")),
    database=os.getenv("NEON_DB"),
    query={"sslmode": os.getenv("NEON_SSLMODE", "require")},
)

engine = create_engine(url, pool_pre_ping=True)

# Test connexion
test_df = pd.read_sql("SELECT now() AS now, version() AS version;", engine)
print(test_df)

                               now  \
0 2025-12-18 09:23:44.887314+00:00   

                                             version  
0  PostgreSQL 17.7 (bdc8956) on aarch64-unknown-l...  


In [15]:
## je selectionne les features pour le ML

START_YEAR = 1990
END_YEAR = 2020

TARGET = "co2_emissions"
FEATURES = [
    "oil_production",
    "gas_production",
    "coal_production",
    "nuclear_electricity",
    "solar_electricity",
    "hydro_electricity",
    "wind_electricity",
]

cols_sql = ",\n  ".join([f'f."{c}"' for c in [TARGET] + FEATURES])

query = f"""
SELECT
  f.country_key,
  f.year::int AS year,
  {cols_sql}
FROM public.fact_energy_environment f
JOIN public.dim_country c ON c.country_key = f.country_key
WHERE c.iso_code NOT LIKE 'OWID_%'
  AND f.year BETWEEN :start AND :end
ORDER BY f.country_key, f.year;
"""

with engine.connect() as conn:
    df = pd.read_sql_query(text(query), conn, params={"start": START_YEAR, "end": END_YEAR})

# target obligatoire
df = df.dropna(subset=[TARGET]).copy()

df.head()

,country_key,year,co2_emissions,oil_production,gas_production,coal_production,nuclear_electricity,solar_electricity,hydro_electricity,wind_electricity
0,1,1990,2.024,0.0,31.940985,0.609598,NaN,NaN,NaN,NaN
1,1,1991,1.914,0.0,3.378293,0.545736,NaN,NaN,NaN,NaN
2,1,1992,1.482,0.0,3.253757,0.046446,NaN,NaN,NaN,NaN
3,1,1993,1.487,0.0,3.253757,0.040640,NaN,NaN,NaN,NaN
4,1,1994,1.454,0.0,3.252375,0.034834,NaN,NaN,NaN,NaN


In [18]:
TRAIN_END = 2015

# flags missing: 1=missing, 0=present
for col in FEATURES:
    df[f"{col}_missing"] = df[col].isna().astype(int)

flag_cols = [f"{c}_missing" for c in FEATURES]
X_cols = FEATURES + flag_cols

# split temporel
train = df[df["year"] <= TRAIN_END].copy()
test  = df[df["year"] >  TRAIN_END].copy()

X_train, y_train = train[X_cols], train[TARGET]
X_test,  y_test  = test[X_cols],  test[TARGET]

print("train:", X_train.shape, "test:", X_test.shape)

# preprocess
# Ridge/ElasticNet: impute mean + scale numeric, passthrough flags
preprocess_linear = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler()),
    ]), FEATURES),
    ("flags", "passthrough", flag_cols),
])

# randomForest: impute mean, passthrough flags (no scaling needed)
preprocess_rf = ColumnTransformer([
    ("num", SimpleImputer(strategy="mean"), FEATURES),
    ("flags", "passthrough", flag_cols),
])

# models
ridge = Pipeline([
    ("prep", preprocess_linear),
    ("ridge", RidgeCV(alphas=np.logspace(-3, 3, 25)))
])

enet = Pipeline([
    ("prep", preprocess_linear),
    ("enet", ElasticNetCV(
        l1_ratio=[0.2, 0.5, 0.8, 1.0],
        cv=5,               
        random_state=42,
        max_iter=50000
    ))
])

rf = Pipeline([
    ("prep", preprocess_rf),
    ("rf", RandomForestRegressor(
        n_estimators=400,
        random_state=42,
        n_jobs=-1
    ))
])

# eval helpers
def eval_model(name, pipe):
    pipe.fit(X_train, y_train)
    pred = pipe.predict(X_test)
    return {
        "model": name,
        "MAE": mean_absolute_error(y_test, pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, pred)),
        "R2": r2_score(y_test, pred),
    }

rows = [
    eval_model("RidgeCV", ridge),
    eval_model("ElasticNetCV", enet),
    eval_model("RandomForest", rf),
]

compare = pd.DataFrame(rows).set_index("model")[["MAE", "RMSE", "R2"]]

# winner par RMSE
winner = compare["RMSE"].idxmin()
winner_row = compare.loc[[winner]].copy()
winner_row.index = [f"WINNER (by RMSE) → {winner}"]

compare_out = pd.concat([compare, winner_row]).round(3)
display(compare_out)

# hyperparams choisis (utile pour l'oral)
ridge.fit(X_train, y_train)
enet.fit(X_train, y_train)
print("Ridge best alpha:", ridge.named_steps["ridge"].alpha_)
print("ElasticNet alpha:", enet.named_steps["enet"].alpha_)
print("ElasticNet l1_ratio:", enet.named_steps["enet"].l1_ratio_)

train: (5280, 14) test: (1030, 14)


,MAE,RMSE,R2
RidgeCV,69.629,233.955,0.921
ElasticNetCV,104.240,204.850,0.940
RandomForest,71.310,225.254,0.927
WINNER (by RMSE) → ElasticNetCV,104.240,204.850,0.940


Ridge best alpha: 0.01778279410038923
ElasticNet alpha: 7.434681340555951
ElasticNet l1_ratio: 1.0
